In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/bike-sharing-demand/sampleSubmission.csv
/kaggle/input/bike-sharing-demand/test.csv
/kaggle/input/bike-sharing-demand/train.csv


# Bike sharing demand 
자전거 수요를 RandomForestRegressor 를 사용하여 예측해 봅시다.  
                                                       _jinjae lee_ 만듬


In [2]:
# data import
train = pd.read_csv('/kaggle/input/bike-sharing-demand/train.csv')
sampleSubmission = pd.read_csv('/kaggle/input/bike-sharing-demand/sampleSubmission.csv')
test = pd.read_csv('/kaggle/input/bike-sharing-demand/test.csv')

train.head()

,datetime,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,casual,registered,count
0,2011-01-01 00:00:00,1,0,0,1,9.84,14.395,81,0.0,3,13,16
1,2011-01-01 01:00:00,1,0,0,1,9.02,13.635,80,0.0,8,32,40
2,2011-01-01 02:00:00,1,0,0,1,9.02,13.635,80,0.0,5,27,32
3,2011-01-01 03:00:00,1,0,0,1,9.84,14.395,75,0.0,3,10,13
4,2011-01-01 04:00:00,1,0,0,1,9.84,14.395,75,0.0,0,1,1


## 상관관계 분석

In [3]:
train.corr()

,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,casual,registered,count
season,1.000000,0.029368,-0.008126,0.008879,0.258689,0.264744,0.190610,-0.147121,0.096758,0.164011,0.163439
holiday,0.029368,1.000000,-0.250491,-0.007074,0.000295,-0.005215,0.001929,0.008409,0.043799,-0.020956,-0.005393
workingday,-0.008126,-0.250491,1.000000,0.033772,0.029966,0.024660,-0.010880,0.013373,-0.319111,0.119460,0.011594
weather,0.008879,-0.007074,0.033772,1.000000,-0.055035,-0.055376,0.406244,0.007261,-0.135918,-0.109340,-0.128655
temp,0.258689,0.000295,0.029966,-0.055035,1.000000,0.984948,-0.064949,-0.017852,0.467097,0.318571,0.394454
atemp,0.264744,-0.005215,0.024660,-0.055376,0.984948,1.000000,-0.043536,-0.057473,0.462067,0.314635,0.389784
humidity,0.190610,0.001929,-0.010880,0.406244,-0.064949,-0.043536,1.000000,-0.318607,-0.348187,-0.265458,-0.317371
windspeed,-0.147121,0.008409,0.013373,0.007261,-0.017852,-0.057473,-0.318607,1.000000,0.092276,0.091052,0.101369
casual,0.096758,0.043799,-0.319111,-0.135918,0.467097,0.462067,-0.348187,0.092276,1.000000,0.497250,0.690414
registered,0.164011,-0.020956,0.119460,-0.109340,0.318571,0.314635,-0.265458,0.091052,0.497250,1.000000,0.970948


## 결측치 확인

결측치 없음

In [4]:
train.isnull().sum()

datetime      0
season        0
holiday       0
workingday    0
weather       0
temp          0
atemp         0
humidity      0
windspeed     0
casual        0
registered    0
count         0
dtype: int64

## 데이터 추가

In [5]:
# datetime 컬럼을 datetime type으로 변경
train['datetime'] = pd.to_datetime(train['datetime'])

# datetime 관련 새로운 feature 생성
train['year'] = train['datetime'].apply(lambda x : x.year)
train['month'] = train['datetime'].apply(lambda x : x.month)
train['day'] = train['datetime'].apply(lambda x : x.day)
train['hour'] = train['datetime'].apply(lambda x : x.hour)

# weather 와 season을  one-hot encording화 
weather=pd.get_dummies(train['weather'],prefix='weather')
train=pd.concat([train,weather],axis=1)

season = pd.get_dummies(train['season'], prefix='season')
train=pd.concat([train, season], axis=1)
train.head()

,datetime,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,casual,...,day,hour,weather_1,weather_2,weather_3,weather_4,season_1,season_2,season_3,season_4
0,2011-01-01 00:00:00,1,0,0,1,9.84,14.395,81,0.0,3,...,1,0,1,0,0,0,1,0,0,0
1,2011-01-01 01:00:00,1,0,0,1,9.02,13.635,80,0.0,8,...,1,1,1,0,0,0,1,0,0,0
2,2011-01-01 02:00:00,1,0,0,1,9.02,13.635,80,0.0,5,...,1,2,1,0,0,0,1,0,0,0
3,2011-01-01 03:00:00,1,0,0,1,9.84,14.395,75,0.0,3,...,1,3,1,0,0,0,1,0,0,0
4,2011-01-01 04:00:00,1,0,0,1,9.84,14.395,75,0.0,0,...,1,4,1,0,0,0,1,0,0,0


In [6]:
# test 데이터 셋에도 동일하게 적용
test['datetime'] = pd.to_datetime(test['datetime'])
test['year'] = test['datetime'].apply(lambda x : x.year)
test['month'] = test['datetime'].apply(lambda x : x.month)
test['day'] = test['datetime'].apply(lambda x : x.day)
test['hour'] = test['datetime'].apply(lambda x : x.hour)

weather=pd.get_dummies(test['weather'],prefix='weather')
test=pd.concat([test,weather],axis=1)

season = pd.get_dummies(test['season'], prefix='season')
test=pd.concat([test, season], axis=1)
test.head()


,datetime,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,year,...,day,hour,weather_1,weather_2,weather_3,weather_4,season_1,season_2,season_3,season_4
0,2011-01-20 00:00:00,1,0,1,1,10.66,11.365,56,26.0027,2011,...,20,0,1,0,0,0,1,0,0,0
1,2011-01-20 01:00:00,1,0,1,1,10.66,13.635,56,0.0000,2011,...,20,1,1,0,0,0,1,0,0,0
2,2011-01-20 02:00:00,1,0,1,1,10.66,13.635,56,0.0000,2011,...,20,2,1,0,0,0,1,0,0,0
3,2011-01-20 03:00:00,1,0,1,1,10.66,12.880,56,11.0014,2011,...,20,3,1,0,0,0,1,0,0,0
4,2011-01-20 04:00:00,1,0,1,1,10.66,12.880,56,11.0014,2011,...,20,4,1,0,0,0,1,0,0,0


In [7]:
train.columns

Index(['datetime', 'season', 'holiday', 'workingday', 'weather', 'temp',
       'atemp', 'humidity', 'windspeed', 'casual', 'registered', 'count',
       'year', 'month', 'day', 'hour', 'weather_1', 'weather_2', 'weather_3',
       'weather_4', 'season_1', 'season_2', 'season_3', 'season_4'],
      dtype='object')

## 데이터 셋 준비
- Train X에 필요 없는 features를 drop
- train y는 count만 사용
- 여기에서는 특별하게 데이터를 drop 하지 않았지만 상관관계나 그래프를 그려 이상치나 필요하지 않은 feature를 제거하면 성능이 올라갈 것이라고 생각됩니다.

In [8]:
train_label = train.columns
droped_train_label = train_label.drop(['datetime', 'season', 'weather', 'casual', 'registered', 'count'])
print(droped_train_label)

X = df[droped_train_label]
y = df['count']
print(X.shape)
print(y.shape)


Index(['holiday', 'workingday', 'temp', 'atemp', 'humidity', 'windspeed',
       'year', 'month', 'day', 'hour', 'weather_1', 'weather_2', 'weather_3',
       'weather_4', 'season_1', 'season_2', 'season_3', 'season_4'],
      dtype='object')


NameError: name 'df' is not defined

## 모델 생성
- RandomForestRegressor 를 사용
- n_estimators: 생성할 Tree의 개수를 뜻함
- random_state: 동일한 모델을 생성할 시에 같은 결과값을 보여주기 위함

In [9]:
from sklearn.ensemble import RandomForestRegressor

rf = RandomForestRegressor(n_estimators= 500, random_state=42)

In [10]:

rf.fit(X, y)

NameError: name 'X' is not defined

In [11]:
# 예측을 해봅시다.
predictions = rf.predict(test_df[droped_train_label])

NameError: name 'test_df' is not defined

## 결과 제출

In [12]:
sampleSubmission['count'] = predictions
display(predictions)
print(sampleSubmission)
sampleSubmission.to_csv('pre0.csv', index=False)

NameError: name 'predictions' is not defined